In [82]:
import pandas as pd
import nltk


In [83]:
df = pd.read_csv('../Data/raw/enron_spam_data.csv', encoding = "ISO-8859-1")
df.sample(5)

,Message ID,Subject,Message,Spam/Ham,Date
1610,1610,"enron / hpl actuals for sept . 13 , 2000",teco tap 10 . 938 / enron,ham,2000-09-14
31781,31781,yearly member breakdown,"been ok lately ? ,\nwhile searching google i f...",spam,2005-02-13
21660,21660,message subject,w . o . w . . . . highhest quality medds at gr...,spam,2005-04-06
457,457,team room,- - - - - - - - - - - - - - - - - - - - - - fo...,ham,2000-03-19
1419,1419,"enron actuals for august 17 , 2000",teco tap 120 . 000 / iferc 20 . 000 / enron\nl...,ham,2000-08-18


In [84]:
df.shape

(33716, 5)

In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33716 entries, 0 to 33715
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Message ID  33716 non-null  int64 
 1   Subject     33427 non-null  object
 2   Message     33345 non-null  object
 3   Spam/Ham    33716 non-null  object
 4   Date        33716 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.3+ MB


In [86]:
#Drop rows with null elements
#Drop Cols: message ID and Date
df.dropna(inplace=True)
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33107 entries, 1 to 33715
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Message ID  33107 non-null  int64 
 1   Subject     33107 non-null  object
 2   Message     33107 non-null  object
 3   Spam/Ham    33107 non-null  object
 4   Date        33107 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.5+ MB


(33107, 5)

In [87]:
 # drop  2 cols
df.drop(columns=['Message ID','Date'],inplace=True)
df.sample(5)

,Subject,Message,Spam/Ham
21819,thanks for yesterady,you have to know that first impressions are so...,spam
5196,revised : organizational changes,to : enron north america corp .\nfrom : cliff ...,ham
31077,super viagra is here,new clalls softtabs = lnstant rockhard erectlo...,spam
4717,"94 % off on software - autodesk , corel , xp ,...","hi ,\nfind lots of cheap software in our site ...",spam
8269,re : ca for henwood engagement,"stinson ,\nplease find attached a revised vers...",ham


In [88]:
# renaming  cols
df.rename(columns={'Subject':'sub','Message':'msg','Spam/Ham':'label'},inplace=True)
df.sample(5)

,sub,msg,label
31323,"delivery confirmation , tracking number : yqf ...",noprepdoc . com\nthe secret of life is honesty...,spam
31623,awaited response . .,from the desk of mr . ibrahim mutiu\nproject i...,spam
14050,re : what ' s in rawhide - urgent,i ' m not familiar with the individual assets ...,ham
4252,celebrate nicolas ' s birthday today,fuller harder\nerectionsstarts working within ...,spam
29683,re : ok,"i was well satisfied with my cabin , which was...",spam


In [89]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

df['label'] = encoder.fit_transform(df['label'])

df.sample(5)

,sub,msg,label
28387,pulse survey results,thank you for your participation in the pulse ...,0
10027,work at home and make money,would you like to get rich working part - time...,1
26987,largest collection of dowlnoadable porn d \ / ...,we have the hottest pornostars pics and videos...,1
26325,"all graphics software available , cheap oem ve...","good morning ,\nwe we offer latest oem package...",1
32147,dirty neighbour teen sohwing d - cup tits,"ciao , baby ! : )\nmechikung\ndarlin ! : )\nka...",1


In [90]:
#search dubs
print("found ", df.duplicated().sum(), "dubs")
df = df.drop_duplicates(keep='first')
print("dubs remain: ",df.duplicated().sum())

found  3071 dubs
dubs remain:  0


In [91]:
df.shape

(30036, 3)

In [92]:
df.head()

,sub,msg,label
1,"vastar resources , inc .","gary , production from the high island larger ...",0
2,calpine daily gas nomination,- calpine daily gas nomination 1 . doc,0
3,re : issue,fyi - see note below - already done .\nstella\...,0
4,meter 7268 nov allocation,fyi .\n- - - - - - - - - - - - - - - - - - - -...,0
5,mcmullen gas for 11 / 99,"jackie ,\nsince the inlet to 3 river plant is ...",0


In [93]:
df['label'].value_counts()

0    15861
1    14175
Name: label, dtype: int64

In [95]:
#merge sub + msg

df['text'] = df['sub'] + " " +  df['msg']
df.drop(columns=['msg','sub'],inplace=True)
df.sample(5)

,label,text
23195,0,test results of eol management report this is ...
13054,0,"board presentation - revised louise ,\nthe mul..."
29635,1,"we sell regalis for an affordable price hi ,\n..."
4678,1,the daily stock barometer investor aiert - l r...
1319,0,"hpl nom for august 2 , 2000 ( see attached fil..."


In [104]:
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()
import string

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

def transform_text(text):
    text = text.lower()
    text = nltk.word_tokenize(text)

    #retaining only numbers and alphabets
    y = []
    for i in text:
        if i.isalpha():
            y.append(i)

    #clone copy of list y
    text = y[:]
    y.clear()

    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)

    text = y[:]
    y.clear()

    #stemming
    for i in text:
        y.append(lemmatizer.lemmatize(i))


    return " ".join(y)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jopihabich/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jopihabich/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/jopihabich/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [105]:
#Test preprocessing
transform_text("I'm gonna be home soon 2 and i don't want to talk about this stuff anymore tonight, k? I've cried enough today.")

'gon na home soon want talk stuff anymore tonight k cried enough today'

In [106]:
df['transformed_text'] = df['text'].apply(transform_text)
df.head()

,label,text,num_characters,num_words,num_sentences,transformed_text
2,0,calpine daily gas nomination - calpine daily g...,67,12,1,calpine daily gas nomination calpine daily gas...
3,0,re : issue fyi - see note below - already done...,1182,347,5,issue fyi see note already done stella forward...
4,0,meter 7268 nov allocation fyi .\n- - - - - - -...,1150,322,6,meter nov allocation fyi forwarded lauri allen...
5,0,"mcmullen gas for 11 / 99 jackie ,\nsince the i...",559,125,5,mcmullen gas jackie since inlet river plant sh...
6,0,"meter 1517 - jan 1999 george ,\ni need the fol...",433,95,1,meter jan george need following done jan zero ...


In [116]:
df_clean = df.drop(columns=['text'])
df = df.dropna(inplace=True)
df.rename(columns={'transformed_text':'text'},inplace=True)
df_clean.to_csv("../Data/preprocessed/enron_preprocessed.csv",index=False)
